# Dependences

In [17]:
include("../../src/struct_data.jl")
include("../../src/neighbor.jl")
include("../../src/forces/forces.jl")
include("../../src/run_event.jl")

RunFusionAggregates (generic function with 1 method)

# Model

In [18]:
@time model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 100000.0,
        dt    = 0.5,
        nₖₙₙ  = 100,
        nₛₐᵥₑ = 50
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_4_MiniTest",
        path_output = ""
    ) 
)

dump(model)

  0.000000 seconds (4 allocations: 144 bytes)
ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 100000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 100
    nₛₐᵥₑ: Int64 50
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String "../../data/init/Sphere"
  Output: OutputModel
    name_output: String "Test_4_MiniTest"
    path_output: String ""


In [19]:
Par1 = Cubic(0.001,2.0,3.0)
Par2 = ContractilePar(0.001)
agg_size = 15

15

# Aggregate

In [20]:
# Run only one aggregate
agg = nothing
agg = Aggregate(
    [AggType(
        "HEK_1", 
        InteractionPar(Par1, Par2),
        Float32.(readdlm("../../data/init/Sphere/$(agg_size).0.xyz")[3:end,2:end]) |> cu
    )], 
    [AggLocation("HEK_1",[0 0 0]),],
    model
)

Aggregate(AggType[AggType("HEK_1", InteractionPar(Cubic{Float64}(0.001, 2.0, 3.0), ContractilePar(0.001)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})], AggIndex([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], ["HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"  …  "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"]), Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], AggGeometry(Float32[15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27  …  15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), AggSimulation(AggParameter(Cubic{CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}}(Float32[0.001, 0.001, 0.001, 0.001, 0.001,

In [21]:
# # Run fusion of two aggregates
# agg = nothing
# agg = FusionAggregate(
#     [AggType(
#         "HEK_1", 
#         InteractionPar(Par1, Par2),
#         Float32.(readdlm("../../data/init/Sphere/$(agg_size).0.xyz")[3:end,2:end]) |> cu
#     )], 
#     model
# )

# kNN

In [22]:
dist = zeros(size(agg.Simulation.Neighbor.idx_red)) |> cu
threads=(100)
@cuda(
    threads=threads,
    blocks=cld.(size(agg.Position,1),threads),
    dist_kernel!(
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Neighbor.idx_cont,
        agg.Simulation.Neighbor.idx_sum,
        dist,
        agg.Position,
        agg.Simulation.Parameter.Force.rₘₐₓ
    )
)   
display(agg.Simulation.Neighbor.idx_sum)
display(agg.Simulation.Neighbor.idx_red)

1×2504 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 9  10  9  9  11  12  12  10  9  12  …  12  10  9  11  12  12  10  9  10  9

1000×2504 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 36  37  38  41  42  43  44  45  57  …  2461  2462  2463  2468  2469  2470
  6   7   8  49   1   2   3   3  58     2502  2503  2504  2498  2499  2500
  5   6   7  50  50   1   2  53  15     2503  2504  2455  2499  2500  2501
 42  43  44   5  51  51  52  54  10     2454  2455  2456  2461  2462  2463
 43  44  45  10   6  52  53   7  49     2453  2454  2500  2460  2461  2462
  2   3   2   9   4   7   8  14   4  …  2498  2499  2495  2503  2502  2503
 35   1  37  40  11   5   6  13  66     2500  2501  2496  2469  2504  2471
 37  36  39  42  10  12  13  44  50     2493  2494  2462  2467  2468  2469
 51  38  53  58  41  11  12  46  48     2494  2495  2464  2453  2470  2455
  0  52   0   0  43  42  43  62   0     2460  2461  2447     0  2454     0
  ⋮                   ⋮              ⋱                 ⋮              
  0   0   0   0   0   0   0   0   0        0     0     0     0     0     0
  0   0   0   0   0   0   0   0   0        0     0  

# Sum Forces

In [23]:
threads=(64,3)
CUDA.@time @cuda(
    threads = threads,
    blocks = cld.(size(agg.Position,),threads),
    shmem=prod(threads.+2)*sizeof(Float32),
    sum_force!(
        agg.Position,
        agg.Simulation.Force.F,
        agg.Simulation.Force.Pol,
        agg.Simulation.Neighbor.idx_sum,
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Parameter.Force,
        agg.Simulation.Parameter.Contractile.fₚ,
        atan(1),
        model.Time.dt
    )
)

display(agg.Simulation.Force.F)
display(agg.Position)

2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  0.0         -2.77401f-5   4.45587f-6
 -1.9804f-6    3.43513f-5   5.26321f-6
  4.45587f-6  -2.92542f-6   2.32253f-5
 -2.29687f-6  -1.61107f-5   2.18539f-5
 -8.9194f-7    1.27399f-5   3.95159f-6
  1.37052f-6   2.79109f-6  -4.34949f-6
  0.0          3.01493f-6   4.48162f-6
  3.24018f-7   2.64352f-5   1.88292f-5
  2.76312f-5  -2.74932f-5   7.45009f-7
  6.65786f-6   2.31785f-6   3.59743f-5
  ⋮                        
 -4.10157f-6   9.98547f-6  -3.64828f-5
  0.0          2.73031f-5  -4.07225f-5
  6.53675f-6  -1.68247f-7  -1.73069f-5
 -4.92937f-6  -2.64649f-5  -3.64654f-5
 -3.04279f-6  -7.56662f-6  -3.73488f-5
  8.9194f-7   -1.49172f-5  -3.64654f-5
 -1.23126f-5   6.1163f-6   -3.64654f-5
  0.0          1.49224f-6  -3.64654f-5
 -2.40643f-5   1.49224f-6  -3.64654f-5

2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.5       -4.62001  -13.88
  0.499999  -4.61998  -13.88
  2.5       -4.62     -13.88
 -4.5       -2.89001  -13.88
 -2.5       -2.88999  -13.88
 -0.499999  -2.89     -13.88
  1.5       -2.89     -13.88
  3.5       -2.88999  -13.88
 -5.49999   -1.15001  -13.88
 -3.5       -1.15     -13.88
  ⋮                   
  4.5        1.15      13.88
 -4.5        2.89001   13.88
 -2.5        2.89      13.88
 -0.500002   2.88999   13.88
  1.5        2.89      13.88
  3.5        2.88999   13.88
 -1.50001    4.62      13.88
  0.5        4.62      13.88
  2.49999    4.62      13.88

  0.378196 seconds (580.03 k CPU allocations: 28.275 MiB, 3.27% gc time)


# MiniTest

In [24]:
run_test(agg, model, "mini Testing")
display(sum(isnan.(agg.Simulation.Force.F), dims=1))
display(sum(agg.Simulation.Force.F .> 50, dims=1))
display(agg.Simulation.Force.F)
display(agg.Position)

mini Testing...   2%|▌                                   |  ETA: 0:00:11

mini Testing...   2%|▉                                   |  ETA: 0:00:12

mini Testing...   3%|█▏                                  |  ETA: 0:00:13

mini Testing...   6%|██▏                                 |  ETA: 0:00:12

mini Testing...   8%|██▉                                 |  ETA: 0:00:12

...

mini Testing... 100%|████████████████████████████████████| Time: 0:00:11


1×3 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 0  0  0

1×3 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 0  0  0

2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -0.000155377  -2.55673f-5  -2.71316f-6
 -6.76747f-6    3.55783f-5  -2.01948f-5
  0.000159723  -2.7743f-5    3.12793f-7
 -8.24873f-5   -1.23097f-5   2.10056f-7
 -4.5201f-5     1.0311f-5    1.47102f-5
  4.44523f-5    1.46441f-7   6.65743f-6
  1.7279f-5    -2.01355f-7   1.49242f-5
  6.36358f-5    2.61349f-5   1.83964f-5
 -8.18957f-5   -5.6303f-6   -3.00538f-5
  3.66105f-5    3.96957f-6   3.4818f-5
  ⋮                         
  0.000106493   9.96357f-6  -3.64691f-5
 -0.000769992   3.38573f-6  -4.07177f-5
  0.000631616  -3.42219f-5  -1.73061f-5
 -1.13618f-5   -2.64682f-5  -3.64493f-5
  9.97823f-5   -3.38653f-6  -3.73351f-5
 -3.51595f-5   -1.48813f-5  -3.64704f-5
 -0.000409421   1.64487f-5  -3.64445f-5
 -1.45562f-5    1.47801f-6  -3.64669f-5
  0.00042534    1.47681f-6  -3.64577f-5

2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.40722   -4.62003  -13.88
  0.477471  -4.61997  -13.88
  2.3605    -4.62002  -13.88
 -4.31336   -2.89001  -13.88
 -2.38783   -2.88999  -13.88
 -0.485665  -2.89     -13.88
  1.43149   -2.89     -13.88
  3.36371   -2.88997  -13.88
 -5.28149   -1.15002  -13.88
 -3.35501   -1.15     -13.88
  ⋮                   
  4.39687    1.15001   13.88
 -4.24611    2.89002   13.88
 -2.28719    2.88998   13.88
 -0.38137    2.88997   13.88
  1.51834    2.88999   13.88
  3.48691    2.88999   13.88
  0.648256   4.62001   13.88
 -1.34051    4.62      13.88
  2.41061    4.62      13.88